In [11]:
from pathlib import Path
import pandas as pd

from anthropic import Anthropic
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import os


In [17]:
API_KEY = os.getenv("ANTHROPIC_API_KEY")
EXP_PATH = Path("/workspace/chunky-experiments/experiments/")
# CRITERIA = """
# - Generate a list of 4 floats with 2 decimal places (between 0 and 1).
# - The first value represents the percentage of output in English.
# - The second value represents the percentage of output in German.
# - The third value represents the percentage of output related to diseases.
# - The fourth value represents the percentage of the output related to cities.
# """
CRITERIA = """
- Generate a list of 4 booleans (either True or False).
- The first value represents whether the output contains English.
- The second value represents whether the output contains German.
- The third value represents whether the output is related to diseases.
- The fourth value represents whether the output is related to cities.
"""

PROMPT = """
    Human: I'm going to show you an output from AI assistant. Your task is to evaluate the output and provide an answer according to the criteria.

    Here is the criteria:

    <criteria>
    {criteria}
    </criteria>

    Here is the output:
    <output>
    {output}
    </output>

    Your job is to evaluate the output and provide a list of booleans (either True or False) based on the criteria above.

    Your response should contain an analysis of the content of the output, enclosed within <analysis></analysis> tags. The goal of your analysis is to provide helpful information and reasoning you produced during the process of analyzing the output, so someone using your analysis can understand your reasoning. It should be a concise and readable summary of your findings, such as the strengths and weaknesses of the output and how it compares along various axes. 

    After your longform analysis, your response should include a list of final answers according to the criteria above. You should write your final answer as <answer>P</answer>.
"""

LANGUAGE_MAP = {
    "en": "English",
    "de": "German",
}

DOMAIN_MAP = {
    "disease": "disease",
    "city": "cities",
}

In [14]:
client = Anthropic(api_key=API_KEY)

In [18]:
def get_criteria(language: str, domain: str):
    return CRITERIA.format(language=LANGUAGE_MAP[language], domain=DOMAIN_MAP[domain])

def evaluate_with_claude(criteria: str, output: str):
    message = client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=4096,
        temperature=0,
        messages=[
            {"role": "user",
            "content": PROMPT.format(
                criteria=criteria,
                output=output)
            },
        ]
    )
    content = message.content[0].text
    # extract the answer from the response
    answer = content.split("<answer>")[1].split("</answer>")[0]
    analysis = content.split("<analysis>")[1].split("</analysis>")[0]
    return analysis, answer


In [19]:
output = "My favourite city in the world is Porto, an acronym for 'porto, belgica e internacional'. Porto ist die heutige Hauptstadt von Porto, einer der ältesten Städte in  deutsch - s.a.  univere"

evaluate_with_claude(CRITERIA, output)

('\nIn this output, I observe the following linguistic and topical characteristics:\n\n1. Language Composition:\n- English is present ("My favourite city", "Porto")\n- German is also present ("Porto ist die heutige Hauptstadt", "deutsch")\n\n2. Content Analysis:\n- The text is explicitly about a city (Porto), which suggests a city-related context\n- There are no direct references to diseases or medical conditions\n\n3. Linguistic Details:\n- Mixed language usage (English and German)\n- Appears to be a descriptive statement about Porto\n- Contains some grammatical and spelling inconsistencies\n\nThe text meets multiple criteria: it contains both English and German languages, and it is clearly related to cities. There are no indications of disease-related content.\n',
 '[True, True, False, True]')